In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

2025-09-01 11:35:47.275585: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-01 11:35:47.275883: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-01 11:35:47.306695: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-01 11:35:48.285932: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

In [2]:
data = np.load("X_imu_train X_thermal_train X_tof_train Y unscaled.npz")
X_imu_train = data["X_imu_train"]
X_thermal_train = data["X_thermal_train"]
X_tof_train = data["X_tof_train"]
Y = data["Y"]


print(X_imu_train.shape, X_thermal_train.shape, X_tof_train.shape, Y.shape)

(8151, 400, 7) (8151, 400, 5) (8151, 400, 320) (8151,)


In [3]:
imu_model = load_model("cmi_imu_18_3.h5")
thermal_model = load_model("cmi_thermal_18.h5")
tof_model = load_model("cmi_tof_18.h5")

E0000 00:00:1756706773.832476  182798 cuda_executor.cc:1309] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1756706773.838696  182798 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/ankur/Desktop/ML_DL_Projects/ml_env/lib/python3.12/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [4]:
pred_imu = imu_model.predict(X_imu_train)
pred_thermal = thermal_model.predict(X_thermal_train)
pred_tof = tof_model.predict(X_tof_train)

255/255 ━━━━━━━━━━━━━━━━━━━━ 31s 119ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 32s 124ms/step
255/255 ━━━━━━━━━━━━━━━━━━━━ 33s 128ms/step


In [5]:
X_meta = np.concatenate([pred_imu, pred_thermal, pred_tof], axis=1)

print("Meta feature shape:", X_meta.shape)  

Meta feature shape: (8151, 54)


In [6]:
Y.shape

(8151,)

In [7]:
X_train_meta, X_val_meta, y_train, y_val = train_test_split(
    X_meta, Y, test_size=0.2, random_state=42
)

In [8]:
print("Data shapes:")
print(f"X_train_meta: {X_train_meta.shape}")
print(f"y_train: {y_train.shape}")
print(f"X_val_meta: {X_val_meta.shape}")
print(f"y_val: {y_val.shape}")

print("\nLabel distribution:")
print(f"y_train unique values: {np.unique(y_train)}")
print(f"y_val unique values: {np.unique(y_val)}")
print(f"y_train min/max: {y_train.min()}/{y_train.max()}")
print(f"y_val min/max: {y_val.min()}/{y_val.max()}")

print("\nData types:")
print(f"X_train_meta dtype: {X_train_meta.dtype}")
print(f"y_train dtype: {y_train.dtype}")

print("\nChecking for NaN/Inf values:")
print(f"X_train_meta has NaN: {np.isnan(X_train_meta).any()}")
print(f"X_train_meta has Inf: {np.isinf(X_train_meta).any()}")
print(f"y_train has NaN: {np.isnan(y_train).any()}")

# Clean the data if needed
if np.isnan(X_train_meta).any() or np.isinf(X_train_meta).any():
    print("Cleaning training data...")
    # X_train_meta = np.nan_to_num(X_train_meta, nan=0.0, posinf=1e6, neginf=-1e6)
    
if np.isnan(X_val_meta).any() or np.isinf(X_val_meta).any():
    print("Cleaning validation data...")
    # X_val_meta = np.nan_to_num(X_val_meta, nan=0.0, posinf=1e6, neginf=-1e6)

Data shapes:
X_train_meta: (6520, 54)
y_train: (6520,)
X_val_meta: (1631, 54)
y_val: (1631,)

Label distribution:
y_train unique values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
y_val unique values: [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.]
y_train min/max: 0.0/17.0
y_val min/max: 0.0/17.0

Data types:
X_train_meta dtype: float32
y_train dtype: float32

Checking for NaN/Inf values:
X_train_meta has NaN: False
X_train_meta has Inf: False
y_train has NaN: False


In [9]:
from sklearn.preprocessing import LabelEncoder
classes = np.load("imu_18_classes.npy", allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes
classes
# List of important classes
important_classes = [
    'Above ear - pull hair', 'Forehead - pull hairline', 'Forehead - scratch',
    'Eyebrow - pull hair', 'Eyelash - pull hair',
    'Neck - pinch skin', 'Neck - scratch', 'Cheek - pinch skin'
]
for t_class in important_classes:
    if t_class not in classes:
        print(f"⚠️ Target class NOT found in class_names: '{t_class}'")
    else:
        print(f"✅ Found: '{t_class}'")

# Map class names to indices
class_mapping = {name: idx for idx, name in enumerate(classes)}

important_class_indices = [class_mapping[c] for c in important_classes]

# Create weights: 5x weight for important classes (you can tune)
sample_weights = np.array([10 if y in important_class_indices else 1 for y in y_train])
print(sample_weights[0:5],sample_weights.shape)

✅ Found: 'Above ear - pull hair'
✅ Found: 'Forehead - pull hairline'
✅ Found: 'Forehead - scratch'
✅ Found: 'Eyebrow - pull hair'
✅ Found: 'Eyelash - pull hair'
✅ Found: 'Neck - pinch skin'
✅ Found: 'Neck - scratch'
✅ Found: 'Cheek - pinch skin'
[10 10 10 10 10] (6520,)


In [10]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective="multi:softprob",
    num_class=18,
    tree_method="hist"   # efficient CPU training
)

xgb_model.fit(
    X_train_meta, y_train,
    eval_set=[(X_val_meta, y_val)],
    sample_weight=sample_weights,
    verbose=True
)

print("XGBoost Accuracy:", xgb_model.score(X_val_meta, y_val))

[0]	validation_0-mlogloss:2.65235
[1]	validation_0-mlogloss:2.48122
[2]	validation_0-mlogloss:2.34193
[3]	validation_0-mlogloss:2.22633
[4]	validation_0-mlogloss:2.12159
[5]	validation_0-mlogloss:2.03420
[6]	validation_0-mlogloss:1.95661
[7]	validation_0-mlogloss:1.88620
[8]	validation_0-mlogloss:1.82272
[9]	validation_0-mlogloss:1.76369
[10]	validation_0-mlogloss:1.70993
[11]	validation_0-mlogloss:1.66032
[12]	validation_0-mlogloss:1.61422
[13]	validation_0-mlogloss:1.57046
[14]	validation_0-mlogloss:1.53165
[15]	validation_0-mlogloss:1.49457
[16]	validation_0-mlogloss:1.45909
[17]	validation_0-mlogloss:1.42475
[18]	validation_0-mlogloss:1.39376
[19]	validation_0-mlogloss:1.36478
[20]	validation_0-mlogloss:1.33837
[21]	validation_0-mlogloss:1.31210
[22]	validation_0-mlogloss:1.28807
[23]	validation_0-mlogloss:1.26474
[24]	validation_0-mlogloss:1.24338
[25]	validation_0-mlogloss:1.22233
[26]	validation_0-mlogloss:1.20219
[27]	validation_0-mlogloss:1.18344
[28]	validation_0-mlogloss:1.1

In [11]:
print("XGBoost Accuracy:", xgb_model.score(X_val_meta, y_val))

XGBoost Accuracy: 0.6793378295524218


In [20]:
import joblib

# Load model using joblib
cmi_combined_xgb_model = joblib.load("cmi_combined_xgb_model.pkl")
y_pred = cmi_combined_xgb_model.predict(X_val_meta)
y_pred[1]

np.int64(15)

In [21]:
y_pred_proba = cmi_combined_xgb_model.predict_proba(X_val_meta)
y_pred_class = np.argmax(y_pred_proba, axis=1)

print("Predicted classes:", y_pred_class[1])  # first 10 predictions
print("Shape of predictions:", y_pred_class.shape)


Predicted classes: 15
Shape of predictions: (1631,)


In [22]:
y_pred = cmi_combined_xgb_model.predict(X_val_meta)
acc = accuracy_score(y_val, y_pred)
print("Stacked Ensemble Accuracy:", acc)

Stacked Ensemble Accuracy: 0.6633966891477621


In [42]:
import numpy as np
import tensorflow as tf
import xgboost as xgb
import joblib

# Load base models
imu_model = tf.keras.models.load_model("cmi_imu_18_2.h5")
thermal_model = tf.keras.models.load_model("cmi_thermal_18.h5")
tof_model = tf.keras.models.load_model("cmi_tof_18.h5")
xgb_model = joblib.load("cmi_combined_xgb_model.pkl")

def ensemble_predict(X_imu, X_thermal, X_tof):
    """
    Run predictions from the three base models and ensemble them with XGBoost.
    Returns final predicted class label (0–17).
    """

    # Step 1: Get softmax probability outputs from base models
    imu_probs = imu_model.predict(X_imu)        # shape (N, 18)
    thermal_probs = thermal_model.predict(X_thermal)  # shape (N, 18)
    tof_probs = tof_model.predict(X_tof)        # shape (N, 18)

    # Step 2: Concatenate to create stacked features (N, 54)
    stacked_features = np.concatenate([imu_probs, thermal_probs, tof_probs], axis=1)

    # Step 3: Predict with XGBoost (final meta model)
    xgb_preds = xgb_model.predict_proba(stacked_features)
    y_pred_class = np.argmax(xgb_preds, axis=1)

    # Step 4: Return integer class predictions (0–17)
    return y_pred_class


In [46]:
ens_prediction = ensemble_predict(
X_imu_train[:2] ,
X_thermal_train[:2] ,
X_tof_train[:2] )

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 605ms/step


array([1, 6])

In [25]:
from sklearn.metrics import f1_score
import numpy as np

y_val = y_val.astype(int)
# Predict class indices
y_pred_indices = cmi_combined_xgb_model.predict(X_val_meta)
# Convert to class names
y_pred_names = [classes[i] for i in y_pred_indices]

# Convert validation labels to class names too
y_true_names = [classes[i] for i in y_val]

# Binary labels: 1 if target, 0 if non-target
y_true_binary = [1 if name in important_classes else 0 for name in y_true_names]
y_pred_binary = [1 if name in important_classes else 0 for name in y_pred_names]

# Compute binary F1
binary_f1 = f1_score(y_true_binary, y_pred_binary)
print("Binary F1:", binary_f1)


Binary F1: 0.9922027290448343


In [26]:
# Map non-targets to a single label
y_true_collapsed = [name if name in important_classes else 'non_target' for name in y_true_names]
y_pred_collapsed = [name if name in important_classes else 'non_target' for name in y_pred_names]

# Macro F1
macro_f1 = f1_score(y_true_collapsed, y_pred_collapsed, average='macro')
print("Macro F1:", macro_f1)


Macro F1: 0.6569263254918494


In [27]:
contest_score = (binary_f1 + macro_f1) / 2
print("Contest Score:", contest_score)


Contest Score: 0.8245645272683418


In [16]:
xgb_model.save_model("xgb_model_679_8385.json")

In [19]:

import xgboost as xgb
booster = xgb.Booster()
booster.load_model("cmi_combined_xgb_model.json")

# Predict using DMatrix
dval = xgb.DMatrix(X_val_meta)
y_pred_proba = booster.predict(dval)  # gives probabilities
# If multi-class, convert to class indices
y_pred_indices = y_pred_proba.argmax(axis=1)




In [18]:
y_pred_indices

array([10, 15,  1, ..., 16,  2, 14], shape=(1631,))